# Advanced Python: Building Scalable Applications

### Day 6: Data Persistence, Web APIs and Profiling Code


In [3]:
import pickle
a = 10
b = [11, 22, 33, 4]
c = {"x": 100, "y": 200, "z": (34, 54, 56)}

with open("pickled.dat", "wb") as outs:
    pickle.dump(a, outs)
    pickle.dump(b, outs)
    pickle.dump(c, outs)

In [4]:
ins = open("pickled.dat", "rb")
ins

<_io.BufferedReader name='pickled.dat'>

In [7]:
pickle.load(ins)

{'x': 100, 'y': 200, 'z': (34, 54, 56)}

In [8]:
ins.close()

In [22]:
import shelve
# shelve module implements a "persistent" dictionary

d = shelve.open("shelved_data")
d

In [11]:
d["name"] = "Guido"

In [12]:
d["role"] = "Founder"

In [13]:
d["place"] = "San Francisco"
d

In [15]:
d["place"], d["role"]

('San Francisco', 'Founder')

In [16]:
"name" in d

True

In [17]:
for k, v in d.items():
    print(k, v)

name Guido
role Founder
place San Francisco


In [18]:
d.close()

In [23]:
d["name"]

'Raymond'

In [37]:
a = 1000_000_000_000_000_000_000_000_100
hash(a)

2292473209843970632

In [39]:
a = "Hello_world"
b = "Hello_world"
print(id(a), id(b))

2208434057200 2208434057200


In [26]:
a = "this is a test string"
b = "this is a test string"
print(id(a), id(b))
a == b
print(hash(a), hash(b))

2208433088640 2208432439040
-571275007076476873 -571275007076476873


In [ ]:
# In regular dictionaries - keys must be hashable object
# However, a shelve object expects a key to be a "str" object


In [65]:
d = { str(v): v*v for v in range(100) }
d["5"]

25

In [66]:
import sys
sys.getsizeof(d)

4696

In [67]:
import shelve
with shelve.open("squares") as s:
    s.update(d)

In [50]:
d = {}
print(sys.getsizeof(d))
d["name"] = "john"
d["role"] = "admin"
d["place"] = "bengaluru"
print(sys.getsizeof(d))
del d["role"]
del d["place"]
print(sys.getsizeof(d))
del d["name"]
print(d)
print(sys.getsizeof(d))




64
232
232
{}
232


In [71]:
import shelve

s = shelve.open("squares")
print(sys.getsizeof(s))
print(s["50"])
print(sys.getsizeof(s))


48
2500
48


#### Working with redis

In [72]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [73]:
import redis

conn = redis.ConnectionPool(host="localhost", port=6379, db=0)
conn

ConnectionPool<Connection<host=localhost,port=6379,db=0>>

In [74]:
d = redis.Redis(connection_pool=conn)
d

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [75]:
d["name"] = "John"

In [76]:
d["name"]

b'Emily'

In [77]:
print(dir(d))

['RESPONSE_CALLBACKS', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__del__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_disconnect_raise', '_eval', '_evalsha', '_fcall', '_georadiusgeneric', '_geosearchgeneric', '_is_protocol', '_is_runtime_protocol', '_send_command_parse_response', '_zaggregate', '_zrange', 'acl_cat', 'acl_deluser', 'acl_dryrun', 'acl_genpass', 'acl_getuser', 'acl_help', 'acl_list', 'acl_load', 'acl_log', 'acl_log_reset', 'acl_save', 'acl_setuser', 'acl_users', 'acl_whoami', 'append', 'auth', 'bf', 'bgrewriteaof', 'bgsave',

In [80]:
"role" in d

True

In [82]:
d["place"] = "Noida"

In [83]:
d["place"]

b'Noida'

In [86]:
d.expire("place", 20)

True

In [89]:
d["place"]

KeyError: 'place'

In [1]:
# Redis simple publish-subscribe example
from redis import Redis
r = Redis() # connects to 'localhost:6379' and uses db=0
print(r)
ps = r.pubsub()
ps
print(ps)

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>


In [94]:
def simple_subscriber(*args, **kwargs):
    print(f"*** got message: {args=}, {kwargs=}")

ps.subscribe(test_event=simple_subscriber)


In [3]:
def simple_subscriber(info):
    print(f"*** got message on channel {info['channel']}, message {info['data']}")
    for k, v in info.items():
        print(f"    {k} => {v}")


ps.subscribe(test_event=simple_subscriber)


In [ ]:
for m in ps.listen():
    print(m)

{'type': 'subscribe', 'pattern': None, 'channel': b'test_event', 'data': 1}
*** got message on channel b'test_event', message b'this is a test message'
    type => message
    pattern => None
    channel => b'test_event'
    data => b'this is a test message'


In [1]:
pip install rq

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Using Redis-Queue for task queues
from redis import Redis
from rq import Queue
q = Queue(connection=Redis())
q

Queue('default')

In [5]:
%cd python_rq

c:\Users\chandrashekar\APBSA\Day_6\python_rq


In [7]:
from simple_tasks import add

In [8]:
add(2, 3)

add method invoked: x=2, y=3


5

In [10]:
r = q.enqueue(add, 2, 3)
r

Job('c086a81d-7333-40f3-9893-93f02cba04ef', enqueued_at=datetime.datetime(2024, 11, 25, 6, 4, 14, 908911, tzinfo=datetime.timezone.utc))

In [15]:
r.get_status()

<JobStatus.FINISHED: 'finished'>

In [18]:
r.result

5

In [19]:
pwd

'c:\\Users\\chandrashekar\\APBSA\\Day_6\\python_rq'

In [20]:
from simple_tasks import add_slow
results = []
for i in range(10):
    print(f"Enqueuing add_slow({i*2}, {i**2})")
    ret = q.enqueue(add_slow, i*2, i**2)
    results.append(ret)


Enqueuing add_slow(0, 0)
Enqueuing add_slow(2, 1)
Enqueuing add_slow(4, 4)
Enqueuing add_slow(6, 9)
Enqueuing add_slow(8, 16)
Enqueuing add_slow(10, 25)
Enqueuing add_slow(12, 36)
Enqueuing add_slow(14, 49)
Enqueuing add_slow(16, 64)
Enqueuing add_slow(18, 81)


In [23]:
for r in results:
    print(r.result)

0
3
8
15
24
35
48
63
80
99


In [25]:
%cd ..

c:\Users\chandrashekar\APBSA\Day_6


In [26]:
import sqlite3 as driver

conn = driver.connect("simple_db")
conn

In [27]:
driver.connect?

Docstring:
connect(database[, timeout, detect_types, isolation_level,
        check_same_thread, factory, cached_statements, uri])

Opens a connection to the SQLite database file *database*. You can use
":memory:" to open a database connection to a database that resides in
RAM instead of on disk.
Type:      builtin_function_or_method


In [28]:
cur = conn.cursor()
cur

In [30]:
cur.execute("""CREATE TABLE users(
     user_id INTEGER PRIMARY KEY AUTOINCREMENT,
     name VARCHAR(32),
     email VARCHAR(32)      
)""")

OperationalError: table users already exists

In [31]:
cur.execute("""INSERT INTO users(name, email) VALUES('john', 'john.doe@gmail.com')""")


In [32]:
user_info = [
    ("emily", "em1234@gmail.com"),
    ("mary", "jane@yahoo.com"),
    ("smith", "smith3344@hotmail.com")
]

INSERT_SQL = """INSERT INTO users(name, email) VALUES(?,?)"""

cur.executemany(INSERT_SQL, user_info)


In [33]:
conn.commit()

In [34]:
conn.close()

In [38]:
import sqlite3 as driver
with driver.connect("simple_db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM users")
    for row in cur:
        print(row)


(1, 'john', 'john.doe@gmail.com')
(2, 'emily', 'em1234@gmail.com')
(4, 'smith', 'smith3344@hotmail.com')


In [37]:
with driver.connect("simple_db") as conn:
    cur = conn.cursor()
    cur.execute("DELETE FROM users WHERE name = ?", ('mary',))
    cur.execute("SELECT * FROM users")
    for row in cur:
        print(row)

(1, 'john', 'john.doe@gmail.com')
(2, 'emily', 'em1234@gmail.com')
(4, 'smith', 'smith3344@hotmail.com')


In [41]:
cd generalized_dbapi_design

c:\Users\chandrashekar\APBSA\Day_6\generalized_dbapi_design


In [ ]:
import sqlite3_config
from user_manager import User

with User(sqlite3_config) as users:
    users.add("james", "wel234", "James Wellington")
    users.add("sam", "sam123", "Samuel Jones")


In [42]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [43]:
from pymongo import MongoClient

In [45]:
MongoClient?

Init signature:
MongoClient(
    host: Union[str, Sequence[str], NoneType] = None,
    port: Optional[int] = None,
    document_class: Optional[Type[~_DocumentType]] = None,
    tz_aware: Optional[bool] = None,
    connect: Optional[bool] = None,
    type_registry: Optional[bson.codec_options.TypeRegistry] = None,
    **kwargs: Any,
) -> None
Docstring:     
A client-side representation of a MongoDB cluster.

Instances can represent either a standalone MongoDB server, a replica
set, or a sharded cluster. Instances of this class are responsible for
maintaining up-to-date state of the cluster, and possibly cache
resources related to this, including background threads for monitoring,
and connection pools.
Init docstring:
Client for a MongoDB instance, a replica set, or a set of mongoses.

The client object is thread-safe and has connection-pooling built in.
If an operation fails because of a network error,
:class:`~pymongo.errors.ConnectionFailure` is raised and the client
reconnects in t

In [46]:
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [47]:
c = client["company"]
c

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'company')

In [48]:
c1 = client.books
c1

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'books')

In [51]:
e = c.employees
r = e.insert_one({"name": "Sam", "role": "Developer"})
r

In [52]:
r.inserted_id

ObjectId('67442d50618a9e5a517a70ad')

In [54]:
data = [
    {"name": "Guido", "role": "Chief"},
    {"name": "Tim", "role": "Maintainer"},
    {"place": "Denver", "event": "PyCon 2025"},
    {"city": "Bengaluru", "event": "PyCon 2025"}
]

r = e.insert_many(data)
r

In [56]:
e.find_one({"name": "Guido"})

{'_id': ObjectId('67442e17618a9e5a517a70ae'), 'name': 'Guido', 'role': 'Chief'}

In [57]:
for doc in e.find({"event": "PyCon 2025"}):
    print(doc)


{'_id': ObjectId('67442e17618a9e5a517a70b0'), 'place': 'Denver', 'event': 'PyCon 2025'}
{'_id': ObjectId('67442e17618a9e5a517a70b1'), 'city': 'Bengaluru', 'event': 'PyCon 2025'}


In [58]:
pip install fastapi

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 2.0/2.0 MB 4.1 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install uvicorn

Note: you may need to restart the kernel to use updated packages.


In [68]:
import httpx

ep = "http://localhost:8000/users"
res = httpx.get(ep)
res.json()

[{'id': '1234', 'name': 'john', 'password': 'john123', 'fullname': 'John Doe'}]

In [66]:
res = httpx.post(ep, json={
      "name": "john", 
      "password": "john123", 
      "fullname": "John Doe",
      "id": "1234"})
res

<Response [200 OK]>

In [67]:
res.json()

In [72]:
res = httpx.patch(ep, params={"name": "john"}, json={
    "fullname": "abcd", 
    "name": "john",
    "password": "welcome",
    "id": "1234"})
res

<Response [405 Method Not Allowed]>

In [70]:
res

<Response [405 Method Not Allowed]>